In [1]:
import cv2
import streamlink
from ultralytics import YOLO
import numpy as np

In [ ]:
import cv2
import streamlink
from ultralytics import YOLO
import numpy as np
import time

# Funktionen för att räkna IoU (Intersection over Union) för trackingen
def iou(box1, box2):
    x1, y1, x2, y2 = box1
    x1g, y1g, x2g, y2g = box2

    xi1 = max(x1, x1g)
    yi1 = max(y1, y1g)
    xi2 = min(x2, x2g)
    yi2 = min(y2, y2g)

    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2g - x1g) * (y2g - y1g)

    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area > 0 else 0

# Laddar in vår YOLO-modell
model = YOLO('./Results/train5/weights/best.pt')

# Laddar in Livestreamen, detta från Mammoth Mountain, då Levi var stängt och ingen åkte :(
youtube_url = "https://www.youtube.com/watch?v=UpfvBbw6kSY&ab_channel=MammothMountain"

#Levi nedan
#youtube_url = "https://www.youtube.com/watch?v=Wr9b5aYA4mI&ab_channel=LeviSkiResort"

# Utnyttjar Streamlink för att ta ut videon från urlen
streams = streamlink.streams(youtube_url)
if not streams or "best" not in streams:
    print("No valid stream found!")
    exit()

stream = streams["best"]
direct_stream_url = stream.url

#Öppnar Video Capture från live streamen
cap = cv2.VideoCapture(direct_stream_url)
if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()

# Objectdetektering för att räkna skidare och snowboardåkare
seen_objects = {}  
next_object_id = 0  # unikt ID
skier_count = 0
snowboarder_count = 0
TIME_THRESHOLD = 0.3  # timer som räknar 0,3 sekunder detekterad innan counter ++1

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    results = model(frame)
    current_time = time.time()
    detected_objects = []

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = box.conf[0].item()
            label = int(box.cls[0].item())  # 0 = Skidåkare, 1 = Snowboardåkare

            if confidence > 0.5:
                # Försöker matcha med ett existerande objekt baserat på IoU
                matched_id = None
                for obj_id, obj_info in seen_objects.items():
                    if iou((x1, y1, x2, y2), obj_info["bbox"]) > 0.5:
                        matched_id = obj_id
                        break

                if matched_id is None:
                    matched_id = next_object_id
                    seen_objects[matched_id] = {"timestamp": current_time, "class": label, "bbox": (x1, y1, x2, y2)}
                    next_object_id += 1

                detected_objects.append(matched_id)

                # Räknar ifall objektet finns i bild med än timer (0,3sek)
                if current_time - seen_objects[matched_id]["timestamp"] > TIME_THRESHOLD:
                    if seen_objects[matched_id]["class"] == 0:  # Skidåkare
                        skier_count += 1
                    else:  # Snowboardåkare
                        snowboarder_count += 1

                    del seen_objects[matched_id]  # radera efter räkning

                # Rita boxen
                color = (255, 0, 0) if label == 0 else (0, 0, 255)  # Röd Snowboardåkare, Blå Skidåkare

                #Skriver ut 'Skier #3' baserat på den nuvarande countern (+1 efter 0,3 sekunder i bild)
                text = f"{f'Skier #{skier_count}' if label == 0 else f'Snowboarder #{snowboarder_count}'}: {confidence:.2f}"
                
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Raderar gamla objekt som inte längre finns i bild
    seen_objects = {obj_id: obj_info for obj_id, obj_info in seen_objects.items() if obj_id in detected_objects}

    # Visa countern i mellan toppen
    cv2.putText(frame, f"Skiers: {skier_count}", (150, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    cv2.putText(frame, f"Snowboarders: {snowboarder_count}", (300, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    #Visa själva streamen
    cv2.imshow("YOLO Ski Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 (no detections), 72.7ms
Speed: 2.8ms preprocess, 72.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 56.2ms
Speed: 2.1ms preprocess, 56.2ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 55.7ms
Speed: 1.8ms preprocess, 55.7ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 53.5ms
Speed: 1.2ms preprocess, 53.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 54.3ms
Speed: 1.1ms preprocess, 54.3ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 52.9ms
Speed: 1.5ms preprocess, 52.9ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 52.1ms
Speed: 1.5ms preprocess, 52.1ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 52.2ms
Speed: 1.3ms preprocess, 52.2ms i

KeyboardInterrupt: 